In [2]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType,StringType, FloatType
import pyspark
from pyspark.sql import SparkSession
import os

In [3]:
accessKeyId = "admin"
secretAccessKey = "password"

In [4]:
spark = (
    SparkSession.builder
    .master('spark://spark-iceberg:7077')
    .appName("Spark Nessie Iceberg Demo")
    .config("spark.driver.memory", "16g")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
                 "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")\
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.5.2,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.96.1')
    .config("spark.hadoop.fs.s3a.access.key", accessKeyId)
    .config("spark.hadoop.fs.s3a.secret.key", secretAccessKey)
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.authentication.type", "NONE")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.warehouse", "s3://warehouse/")
    .config("fs.s3a.connection.ssl.enabled", "false")
    .getOrCreate()
)


25/10/04 18:33:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
spark.sql("CREATE DATABASE IF NOT EXISTS test_db")
spark.sql("DROP TABLE IF EXISTS test_db.test_table_001")

DataFrame[]

In [5]:
# Extract a Data
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-04-01 00:00:18|  2021-04-01 00:21:54|            1.0|          8.4|       1.0|                 N|          79|         116|           1|       25.5|  3.0|    0.5|      5.8

In [7]:
df.write.mode("overwrite").format("parquet").saveAsTable("test_db.test_table_001")

In [9]:
# Test Nessie
# List tables
spark.sql("SHOW TABLES IN nessie.default").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [ ]:
spark.stop()